In [14]:
# set parameters
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:,.2f}'.format
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import dmg5e
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn', 'Date': '2021-12-27'}
SAVEFIGS = False
SAVETABS = False

In [15]:
# construct book dataframe
dfB = pd.read_csv('../../assets/data/books.csv')
dfB = dfB.astype({
    'Book': 'category', 
    'Acronym': 'category',
    'Date': 'datetime64',
    'Type': 'category'})
dfB.set_index('Acronym', inplace=True)
dfB

,Book,Date,Type,Status,Added,Processed,Remaining,Published
Acronym,,,,,,,,
BGDiA,Baulder's Gate: Descent Into Avernus,2019-09-17,Adventure,Done,36,36,0,2019-09-17
CM,Candlekeep Mysteries,2021-03-16,Adventure,Done,23,23,0,2021-03-16
CRCotN,Critical Role: Call of the Netherdeep,2022-03-15,Adventure,Done,40,40,0,2022-03-15
CoS,Curse of Strahd,2016-03-16,Adventure,Done,24,24,0,2016-03-16
DSotDQ,Dragonlance: Shadow of the Dragon Queen,2022-12-06,Adventure,NaN,25,0,25,2022-12-06
ERftLW,Eberron: Rising from the Last War,2019-11-19,Source,Done,38,38,0,2019-11-19
EGtW,Explorer's Guide to Wildmount,2020-03-07,Source,Done,34,34,0,2020-03-07
FToD,Fizban's Treasury of Dragons,2021-10-26,Source,Needs mythic creatures and lair actions,104,104,0,2021-10-26
GoS,Ghosts of Saltmarsh,2019-05-21,Adventure,Done,48,48,0,2019-05-21


In [16]:
#from scipy.interpolate import interp1d

# import data
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD.apply(lambda row: estats5e.effHP(row['HP Mean'], row['AC']), axis=1)
dfD['eDPR Mean'] = dfD.apply(lambda row: estats5e.effDPR(row['DPR Mean'], row['AB']), axis=1)
dfD['eXP Mean']  = dfD.apply(lambda row: estats5e.effXP(row['HP Mean'], row['AC'], row['DPR Mean'], row['AB']), axis=1)
dfD.set_index('CR', inplace=True)

#df0 = pd.read_csv('../../assets/data/monsters.csv') 
df0 = pd.read_csv('./monster-cr-and-xp.csv') 
df0 = df0[df0['adj DPR'].gt(0)] 
# 'Book','Page','Type','Category','Legendary','CR','PB','XP',
# 'HP','AC','adj HP','adj AC',
# 'AB','adj AB','DPR', 'adj DPR'

df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category'})

df0['eHP']  = df0.apply(lambda row: estats5e.effHP(row['adj HP'], row['adj AC']), axis=1)
df0['eDPR'] = df0.apply(lambda row: estats5e.effDPR(row['adj DPR'], row['adj AB']), axis=1)
df0['eXP']  = df0.apply(lambda row: estats5e.effXP(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB']), axis=1)
df0['eHP Ratio'] = df0.apply(lambda row: row['eHP']/dfD.loc[row['CR'], 'eHP Mean'], axis=1)
df0['eDPR Ratio'] = df0.apply(lambda row: row['eDPR']/dfD.loc[row['CR'], 'eDPR Mean'], axis=1)
df0['eXP Ratio'] = df0['eXP'] / df0['XP']
df0['eXP Skew'] = (df0['eDPR Ratio'] - df0['eHP Ratio'])/(df0['eDPR Ratio'] + df0['eHP Ratio'])

df0['D-CR DMG'] = df0.apply(lambda row: dmg5e.monster_defensive_challenge_rating(row['adj HP'], row['adj AC']), axis=1)
df0['O-CR DMG'] = df0.apply(lambda row: dmg5e.monster_offensive_challenge_rating(row['adj DPR'], row['adj AB']), axis=1)
df0['C-CR DMG'] = df0.apply(lambda row: np.floor(dmg5e.monster_challenge_rating(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB'])), axis=1)
df0['C-CR DMG Delta'] = df0['C-CR DMG'] - df0['CR']
df0['XP DMG'] = df0.apply(lambda row: dfD.loc[row['C-CR DMG'], 'XP'], axis=1)
df0['XP DMG Ratio'] = df0['XP DMG'] / df0['XP']

df0['Date'] = df0.apply(lambda row: dfB.loc[row['Book'], 'Date'], axis=1)
df0['Book Type'] = df0.apply(lambda row: dfB.loc[row['Book'], 'Type'], axis=1)

len(df0.index.tolist())
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1779 monsters in database


In [17]:
# plot eXP ratio vs time. Split data by book type.
from datetime import date

groups = {
    'adventure modules - normal': {
        'Legendary': ['N'],
        'marker': dict(
            size=10, 
            color='rgba(0,0,0,0.0)', 
            line=dict(color='rgba(31,119,180,1.0)', width=2)
        )
    },
    'adventure modules - legendary': {
        'Legendary': ['L','LL'],
        'marker': dict(
            size=10, 
            color='rgba(31,119,180,1.0)', 
            line=dict(color='rgba(31,119,180,1.0)', width=2)
        )
    }
}

group_columns = ['Date','group','Book']
date_range = [date(2014, 1, 1), date.today()]

# plot results
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        hoverlabel=dict(
            bgcolor='white',
        ),
        xaxis=dict(
            title_text='publication year',
            range=date_range,
            dtick="M12",
            minor=dict(dtick="M3"),
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 3.5],
            tick0=0, dtick=0.5,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='right', x=0.99,
            orientation='v',
        ),
    )
)

for g in groups:
    group = groups[g]
    # summarize the current dataset
    df1 = df0[df0['CR'].between(1,30) & df0['Legendary'].isin(group['Legendary']) & df0['Book Type'].isin(['Adventure'])]
    df1['group'] = df1.apply(lambda row: 'adventure modules - normal' if row['Legendary'] == 'N' else 'adventure modules - legendary', axis=1)
    df1 = df1[group_columns + ['eXP Ratio']]

    dfG = df1.groupby(group_columns).size().to_frame('Total').reset_index()
    dfG['eXP Ratio Mean'] = df1[group_columns + ['eXP Ratio']].groupby(group_columns).mean().reset_index()['eXP Ratio']
    dfG['eXP Ratio Std Dev'] = df1[group_columns + ['eXP Ratio']].groupby(group_columns).std().reset_index()['eXP Ratio']
    dfG = dfG.sort_values(by=['Date'])
    dfG = dfG[dfG['Total'].ge(1)]
    
    fig.add_trace(go.Scatter(
        x=dfG['Date'], 
        y=dfG['eXP Ratio Mean'],
        mode='markers',
        name=g,
    ))

    fig.update_traces(
        selector=dict(name=g),
        customdata = np.stack((dfG['Book'], dfG['Total']), axis=-1),
        hovertemplate = '<b>%{customdata[0]}</b><br>'
            + 'Date: %{x}<br>'
            + 'XP ratio: %{y:.2f}<br>'
            + 'Monsters: %{customdata[1]:.0f}'
            + '<extra></extra>',
        marker=group['marker'],
    )

fig.add_trace(go.Scatter(
    x=date_range, 
    y=[1, 1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-mean-vs-time-adventure-modules-large', style='aspect-ratio: 600/550;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-mean-vs-time-adventure-modules-small', style='aspect-ratio: 600/550;')

In [18]:
# summary table of legendary monsters from early adventure books
df1 = df0[df0['CR'].between(1,30) & df0['Book Type'].isin(['Adventure']) & df0['Date'].lt('2018-1-1') & df0['Legendary'].isin(['L','LL'])]
df1 = df1.sort_values(by=['Date','Monster'])
dfG = df1[['Date','Book','Monster']]
dfG['Monsters'] = df1.groupby(['Book'])['Monster'].transform(lambda x: ', '.join(x))
dfG = dfG[['Book','Monsters']].drop_duplicates()

if SAVETABS: dfG.to_html('./early-adventure-legendary-monsters.html', index=False, classes='center', float_format='{:,.2f}'.format, border=0)

In [19]:
# plot the range of eXP values for legendary monsters encountered directly and indirectly
directMonsters = ['Rezmir','Yestabrod','Tarul Var','Atropal']
df1 = df0[df0['CR'].between(1,30) & df0['Book Type'].isin(['Adventure']) & df0['Date'].lt('2018-1-1') & df0['Legendary'].isin(['L','LL'])]
df1['Encounter Type'] = 'indirect'
df1.loc[df1['Monster'].isin(directMonsters), 'Encounter Type'] = 'direct'
df1.sort_values(by=['Encounter Type','eXP Ratio'], inplace=True)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='encounter type',
            autorange=True,
            
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 4.0],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        ),
    )
)

for x in df1['Encounter Type'].unique():
    fig.add_trace(go.Box(
        y=df1[df1['Encounter Type'].isin([x])]['eXP Ratio'],
        name=x,
        customdata=df1[df1['Encounter Type'].isin([x])]['Monster'],
        showlegend=False,
        boxpoints='all',
        jitter=0.4,
        whiskerwidth=0.4,
        marker_size=8,
        hoveron='points',
        hovertemplate=f'<b>{x}</b><br>'+
            '%{customdata}<br>'+
            'XP ratio %{y:.2f}<br>'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-vs-encounter-type-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-vs-encounter-type-small', style='aspect-ratio: 600/550;')

In [20]:
# list of monsters published in two different books
df1 = df0[df0['CR'].between(1,30) & df0['Legendary'].isin(['LL'])]
dfG = df1.groupby(['Monster','Book']).size().to_frame('Total').reset_index()
dfG = dfG[dfG['Total'].gt(0)]
dfG = dfG.groupby(['Monster']).size().to_frame('Total').reset_index()
dfG = dfG[dfG['Total'].gt(2)]
dupMonsters = dfG['Monster'].tolist()

df1 = df1[df1['Monster'].isin(dupMonsters)]

#books = list(df1['Book'].unique())
books = ['OotA', 'MToF', 'MPMotM']
markers = [
    dict(
        color='#1f77b4', 
        line=dict(color='#1f77b4', width=2),
        size=10,
        symbol='circle',
    ),
    dict(
        color='rgba(0,0,0,0.0)', 
        line=dict(color='#1f77b4', width=2),
        size=10, 
        symbol='circle',
    ),
    dict(
        color='rgba(0,0,0,0.0)', 
        line=dict(color='#1f77b4', width=2),
        size=10, 
        symbol='triangle-up',
    )
]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=80, t=20, pad=4),
        xaxis=dict(
            range=[-0.5, len(dupMonsters)-0.5],
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 2.5],
            tick0=0, dtick=0.5,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='right', x=0.99,
            orientation='v',
        ),
    )
)

# plot data
for book, marker in zip(books, markers):
    data = df1[df1['Book'].isin([book])]
    xpr_mean = np.mean(data['eXP Ratio'])
    xpr_std = np.std(data['eXP Ratio'])
    print(f'{book}: {xpr_mean:.2f} ({xpr_std:.2f})')
    text = [f'{x:.0f}' for x in data['CR'].to_list()]
    fig.add_trace(go.Scatter(
        x=data['Monster'], 
        y=data['eXP Ratio'],
        mode='markers',
        marker=marker,
        name=book,
        text=text,
        hovertemplate=
            f'<b>Book:</b> {book}<br>'+
            '<b>Monster:</b> %{x}<br>'+
            '<b>CR:</b> %{text}<br>'+
            '<b>XP ratio:</b> %{y:.2f}'+
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-vs-monster-demon-lords-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-vs-monster-demon-lords-small', style='aspect-ratio: 600/550;')

OotA: 1.36 (0.31)
MToF: 1.02 (0.19)
MPMotM: 1.00 (0.15)


In [21]:
# plot eXP ratio vs time. Split data by book type.
from datetime import date

groups = {
    'source books - normal': {
        'Legendary': ['N'],
        'marker': dict(
            size=10, 
            color='rgba(0,0,0,0.0)', 
            line=dict(color='#ff7f0e', width=2)
        )
    },
    'source books - legendary': {
        'Legendary': ['L','LL'],
        'marker': dict(
            size=10, 
            color='#ff7f0e', 
            line=dict(color='#ff7f0e', width=2)
        )
    }
}

# summarize the current dataset


group_columns = ['Date','group','Book']
date_range = [date(2014, 1, 1), date.today()]

# plot results
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        hoverlabel=dict(
            bgcolor='white',
        ),
        xaxis=dict(
            title_text='publication year',
            range=date_range,
            dtick="M12",
            minor=dict(dtick="M3"),
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 2],
            tick0=0, dtick=0.5,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='bottom',  y=0.00,
            xanchor='left', x=0.00,
            orientation='v',
        ),
    )
)

for g in groups:
    group = groups[g]
    df1 = df0[df0['CR'].between(1,30) & df0['Legendary'].isin(group['Legendary']) & df0['Book Type'].isin(['Source'])]
    df1['group'] = df1.apply(lambda row: 'source books - normal' if row['Legendary'] == 'N' else 'source books - legendary', axis=1)
    df1 = df1[group_columns + ['eXP Ratio']]
    
    dfG = df1.groupby(group_columns).size().to_frame('Total').reset_index()
    dfG['eXP Ratio Mean'] = df1.groupby(group_columns).mean().reset_index()['eXP Ratio']
    dfG['eXP Ratio Std Dev'] = df1.groupby(group_columns).std().reset_index()['eXP Ratio']
    dfG = dfG.sort_values(by=['Date'])
    dfG = dfG[dfG['Total'].ge(1)]
    
    fig.add_trace(go.Scatter(
        x=dfG['Date'], 
        y=dfG['eXP Ratio Mean'],
        mode='markers',
        name=g,
    ))

    fig.update_traces(
        selector=dict(name=g),
        customdata = np.stack((dfG['Book'], dfG['Total']), axis=-1),
        hovertemplate = '<b>%{customdata[0]}</b><br>'
            + 'Date: %{x}<br>'
            + 'XP ratio: %{y:.2f}<br>'
            + 'Monsters: %{customdata[1]:.0f}'
            + '<extra></extra>',
        marker=group['marker'],
    )

fig.add_trace(go.Scatter(
    x=date_range, 
    y=[1, 1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-mean-vs-time-source-books-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-mean-vs-time-source-books-small', style='aspect-ratio: 600/550;')

In [22]:
# summarize the current dataset
df1 = df0[df0['CR'].between(1,30)  
    & df0['Book Type'].isin(['Source']) 
    & df0['Legendary'].isin(['L','LL'])]

col = 'eXP Ratio'

groups = {
    'MM': {'Book': ['MM'], 'Exclude Type': []},
    'non-MM': {'Book': ['VGtM','MToF','GGtR','ERftLW','EGtW','MOoT','VRGtR','FToD','SaCoC'], 'Exclude Type': []},
    'MM - no dragons': {'Book': ['MM'], 'Exclude Type': ['dragon']},
}

# plot results
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='calculated XP ratio',
            range=[0.5,2.5],
            tick0=0, dtick=0.5, tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        yaxis=dict(
            title_text='CDF',
            range=[-0.01, 1.01],
            tick0=0, dtick=0.2,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=0.00,
            xanchor='right', x=1.00,
            orientation='v',
        ),
    )
)

for g in groups:
    x = df1[df1['Book'].isin(groups[g]['Book']) & ~df1['Type'].isin(groups[g]['Exclude Type'])]['eXP Ratio'].to_numpy()
    x = np.append(np.insert(np.sort(x), 0, 0.0), 3.0)
    y = np.array(range(0,len(x)))/(len(x)-2)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        mode='lines',
        name=g,
        line_shape='hv',
        hovertemplate = '<b>'+g+'</b><br>'
            + 'XP ratio %{y:.2f}<br>'
            + 'CDF %{y:.2f}'
            + '<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-cdf-mm-vs-source-books-no-dragons-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-cdf-mm-vs-source-books-no-dragons-small', style='aspect-ratio: 600/550;')

In [23]:
# plot CDF of eXP Ratio for different source books
plotColumn = 'eXP Ratio'
groups = {
    'MM': {'Book': ['MM'], 'color': '#1f77b4'},
    'non-MM': {'Book': ['VGtM','MToF','GGtR','ERftLW','EGtW','MOoT','VRGtR','FToD','SaCoC'], 'color': '#ff7f0e'}
}

df1 = df0[df0['CR'].between(1,30) & df0['Book Type'].isin(['Source']) & df0['Legendary'].isin(['L','LL'])]
df1['Book'] = df1['Book'].cat.remove_unused_categories()


rMin = 0.5; rMax = 3; rDel = 0.05
rLims = [rMin - rDel/2, rMax + rDel/2]
rRange = np.linspace(rMin - rDel/2, rMax + rDel/2, int(round((rMax - rMin)/rDel)) + 2)

# plot results
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='calculated XP ratio',
            range=[0.5,2.5],
            tick0=0, dtick=0.5, tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        yaxis=dict(
            title_text='CDF',
            range=[-0.01, 1.01],
            tick0=0, dtick=0.2,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=0.00,
            xanchor='right', x=1.00,
            orientation='v',
        ),
    )
)

for b in groups['non-MM']['Book']:
    x = df1[df1['Book'].isin([b])][plotColumn].to_numpy()
    if len(x) == 0:
        continue
    x = np.append(np.insert(np.sort(x), 0, 0.0), 3.0)
    y = np.array(range(0,len(x)))/(len(x)-2)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        mode='lines',
        name=b,
        legendgroup='non-MM',
        showlegend=False,
        line_shape='hv',
        line_width=0.5,
        line_color=groups['non-MM']['color'],
        hovertemplate = '<b>'+b+'</b><br>'
            + 'XP ratio %{y:.2f}<br>'
            + 'CDF %{y:.2f}'
            + '<extra></extra>',
    ))

for g in groups:
    x = df1[df1['Book'].isin(groups[g]['Book'])][plotColumn].to_numpy()
    x = np.append(np.insert(np.sort(x), 0, 0.0), 3.0)
    y = np.array(range(0,len(x)))/(len(x)-2)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        mode='lines',
        name=g,
        legendgroup=g,
        line_shape='hv',
        line_width=2.0,
        line_color=groups[g]['color'],
        hovertemplate = '<b>'+g+'</b><br>'
            + 'XP ratio %{y:.2f}<br>'
            + 'CDF %{y:.2f}'
            + '<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-cdf-mm-vs-source-books-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-cdf-mm-vs-source-books-small', style='aspect-ratio: 600/550;')

In [24]:
# dragons in the MM vs non-dragons
from datetime import date
from bs4 import BeautifulSoup

# summarize the current dataset
df1 = df0[df0['Book'].isin(['MM']) & df0['Legendary'].isin(['L','LL'])]
indx = df1['Type'].isin(['dragon'])

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='CR',
            range=[9.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 2.5],
            tick0=0, dtick=0.5,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='bottom',  y=0.00,
            xanchor='left', x=0.00,
            orientation='v',
        ),
    )
)

# plot results
dft = df1[~indx]
fig.add_trace(go.Scatter(
    x=dft['CR'], 
    y=dft['eXP Ratio'],
    mode='markers',
    name='non-dragons',
    marker=dict(
        size=8, 
        color='#1f77b4', 
        line=dict(color='#1f77b4', width=2)
    ),
    customdata = np.stack((dft['Monster'], dft['Book']), axis=-1),
    hovertemplate = '<b>%{customdata[0]}</b><br>'
                  + 'Book: %{customdata[1]}<br>'
                  + 'CR: %{x:.0f}<br>'
                  + 'XP ratio: %{y:.2f}<br>'
                  + '<extra></extra>',
))

dft = df1[indx]
fig.add_trace(go.Scatter(
    x=dft['CR'], 
    y=dft['eXP Ratio'],
    mode='markers',
    name='dragons',
    marker=dict(
        size=8, 
        color='#ff7f0e', 
        line=dict(color='#ff7f0e', width=2)
    ),
    customdata = np.stack((dft['Monster'], dft['Book']), axis=-1),
    hovertemplate = '<b>%{customdata[0]}</b><br>'
                  + 'Book: %{customdata[1]}<br>'
                  + 'CR: %{x:.0f}<br>'
                  + 'XP ratio: %{y:.2f}<br>'
                  + '<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=[10,30], 
    y=[1, 1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-vs-cr-mm-type-dragon-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-vs-cr-mm-type-dragon-small', style='aspect-ratio: 600/550;')

In [25]:
# comparison between dragons published in the MM and those puliblished in FToD

# summarize the current dataset
df1 = df0[df0['CR'].between(1,30) & df0['Legendary'].isin(['L','LL']) & df0['Type'].isin(['dragon'])]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='CR',
            range=[9.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated XP ratio',
            range=[0, 2],
            tick0=0, dtick=0.5,
            tickformat='.1f',
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='bottom',  y=0.00,
            xanchor='left', x=0.00,
            orientation='v',
        ),
    )
)

# plot results
dft = df1[df1['Book'].isin(['MM'])]
fig.add_trace(go.Scatter(
    x=dft['CR'], 
    y=dft['eXP Ratio'],
    mode='markers',
    name='MM',
    marker=dict(
        size=8, 
        color='#1f77b4', 
        line=dict(color='#1f77b4', width=2)
    ),
    customdata = np.stack((dft['Monster'], dft['Book']), axis=-1),
    hovertemplate = '<b>%{customdata[0]}</b><br>'
                  + 'Book: %{customdata[1]}<br>'
                  + 'CR: %{x:.0f}<br>'
                  + 'XP ratio: %{y:.2f}<br>'
                  + '<extra></extra>',
))

dft = df1[df1['Book'].isin(['FToD'])]
fig.add_trace(go.Scatter(
    x=dft['CR'], 
    y=dft['eXP Ratio'],
    mode='markers',
    name='FToD',
    marker=dict(
        size=8, 
        color='#ff7f0e', 
        line=dict(color='#ff7f0e', width=2)
    ),
    customdata = np.stack((dft['Monster'], dft['Book']), axis=-1),
    hovertemplate = '<b>%{customdata[0]}</b><br>'
                  + 'Book: %{customdata[1]}<br>'
                  + 'CR: %{x:.0f}<br>'
                  + 'XP ratio: %{y:.2f}<br>'
                  + '<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=[10,30], 
    y=[1, 1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-exp-ratio-vs-cr-mm-ftod-type-dragon-large', style='aspect-ratio: 600/500;')
    tfb.save_fig_html(fig, format='small', name='fig-exp-ratio-vs-cr-mm-ftod-type-dragon-small', style='aspect-ratio: 600/550;')